# LSTM_Exam Implementation 

In [3]:
import torch 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('daily_csv.csv')
df = df.dropna()
prices = df['Price'].values
minn, maxx = prices.min(), prices.max()
prices = (prices - minn) / (maxx - minn)
X, y = [], []
sequence_length = 10
for i in range(5900):
    l = []
    for j in range(i, i + sequence_length):
        l.append(prices[j])
    X.append(l)
    y.append(prices[i + sequence_length])
X, y = np.array(X), np.array(y)

In [5]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size= 0.10, random_state= 42, stratify= None)

In [6]:
class Data(torch.utils.data.Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x, dtype= torch.float32)
        self.y = torch.tensor(y, dtype= torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [7]:
train = Data(xtrain, ytrain)
test = Data(xtest, ytest)
train_loader = torch.utils.data.DataLoader(train, batch_size= 256, shuffle= True, drop_last= True)
test_loader = torch.utils.data.DataLoader(test, batch_size= 256, drop_last= True)

### LSTM Model ###

In [8]:
class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size= 1, hidden_size= 5, batch_first= True, num_layers= 1)
        self.fc1 = torch.nn.Linear(in_features= 5, out_features= 1)
    def forward(self, x):
        y, h = self.lstm(x)
        y = y[:, -1, :]
        return self.fc1(torch.relu(y))

In [ ]:
model = Network()
model = model.to('cpu')
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001, weight_decay= 0.0001)

AssertionError: Torch not compiled with CUDA enabled